In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('monthly-milk-production.csv',index_col='Month')
df.index = pd.to_datetime(df.index)
df.head(2)
df.isnull().sum()
df.info()
scalar = MinMaxScaler()
Train = df.head(156)
Test = df.tail(12)
Train_Scalar = scalar.fit_transform(Train)
Test_Scalar = scalar.fit_transform(Test)

In [ ]:
def next_batch(training_data, batch_size, steps):
    
    # Grab a random starting point for each batch
    rand_start = np.random.randint(0,len(training_data)-steps) 

    # Create Y data for time series in the batches
    y_batch = np.array(training_data[rand_start:rand_start+steps+1]).reshape(1,steps+1)
    
    x_batch = y_batch[:, :-1].reshape(-1, steps, 1)    # Date  Shape Batch Timestep inputdim
    y_batch = y_batch[:, 1:].reshape(-1, steps, 1)     # Price STock
    return x_batch , y_batch 

In [ ]:
num_inputs = 1
num_time_steps = 12
num_neurons = 100
num_outputs = 1
learning_rate = 0.01 
num_train_iterations = 4000
batch_size = 1

X = tf.placeholder(tf.float32, [None, num_time_steps, num_inputs])
y = tf.placeholder(tf.float32, [None, num_time_steps, num_outputs])


# Also play around with GRUCell
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.GRUCell(num_units=num_neurons, activation=tf.nn.relu),
    output_size=num_outputs)

output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

loss = tf.reduce_mean(tf.square(output - y)) # MSE
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()

saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_train_iterations):
        
        X_batch, y_batch = next_batch(Train_Scalar, batch_size,num_time_steps)
        sess.run(train, feed_dict={X: X_batch, y: y_batch})
        
        if iteration % 100 == 0:
            
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
        saver.save(sess, "./ex_time_series_soumil")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./ex_time_series_soumil")
    train_seed = list(Train_Scalar[-12:])

    for iteration in range(12):
        X_batch = np.array(train_seed[-num_time_steps:]).reshape(1, num_time_steps, 1)
        y_pred = sess.run(output, feed_dict={X: X_batch})
        train_seed.append(y_pred[0, -1, 0])

In [ ]:
results = scalar.inverse_transform(np.array(train_seed[12:]).reshape(12,1))
results
Test["Predicted"]  = results
Test

In [ ]:
Test.plot()